In [28]:
import shap
import pandas as pd
import pickle
import copy
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import  train_test_split
from sklearn.feature_selection import SelectFromModel
from alibi.confidence import TrustScore
import warnings
warnings.filterwarnings("ignore")
import json

In [2]:
# load the failures data from util
import sys
import os
sys.path.append(os.path.abspath('/Users/supriyasindigerekumaraswmamy/Desktop/Thesis/wind_Turbine'))

from utils.helper import *
failures = load_failures_data('../data/model_data/failures.csv')
components = failures['component'].unique()
component_data = load_all_component_data(components)
data_splits = prepare_all_data_for_training(component_data, "target_class")
models = load_all_models(components, "xgb")
selected_features_data = fit_and_select_features(models, data_splits)
models = retrain_models_on_selected_features(models, selected_features_data, data_splits)

In [3]:
classes = models['GEARBOX'].classes_; len(classes)

2

In [4]:
for component in components:
    globals()[f"{component}_ts"] = TrustScore(alpha=.05,
                filter_type='distance_knn',
                k_filter=10,
                leaf_size=40,
                metric='euclidean',
                dist_filter_type='point')
    globals()[f"{component}_ts"].fit(selected_features_data[component][0].to_numpy(), data_splits[component][2].to_numpy(), classes=len(classes))
    globals()[f"{component}_scores"] = globals()[f"{component}_ts"].score(selected_features_data[component][1].to_numpy(), data_splits[component][3].to_numpy())



In [9]:
def detect_and_replace_outliers(arr, threshold):
    inlier_array = []
    outlier_array = []
    mean_val = np.mean(arr)
    std_dev = np.std(arr)
    upper_threshold = mean_val + threshold * std_dev

    for i, val in enumerate(arr):
        if val > upper_threshold:
            outlier_array.append({"index": i, "value": arr[i]})
        else:
            inlier_array.append({"index": i, "value": arr[i]})

    return inlier_array, outlier_array

In [10]:
for component in components:
   globals()[f"{component}_normal"], globals()[f"{component}_extremes"] = detect_and_replace_outliers(globals()[f"{component}_scores"][0].tolist(), 3)


# Save Trust Score as JSON

In [31]:
normal_TrustScores = []
for component in components:
    for elem in globals()[f"{component}_normal"]:
        TrustScores = {
        "Trust_score" : elem["value"],
        "index" :elem["index"],
        "component" : component
        }
        normal_TrustScores.append(TrustScores)


extreme_TrustScores = []
for component in components:
    for elem in globals()[f"{component}_extremes"]:
        TrustScores = {
        "Trust_score" : elem["value"],
        "index" :elem["index"],
        "component" : component
        }
        extreme_TrustScores.append(TrustScores)


with open('JSON/normal_TrustScores.json', 'w') as json_file:
    json.dump(normal_TrustScores, json_file, indent=4)

with open('JSON/extreme_TrustScores.json', 'w') as json_file:
    json.dump(extreme_TrustScores, json_file, indent=4)



        

